In [1]:
from __future__ import print_function
import sys
import numpy as np
import tables as h5
#import pandas as pd
#pd.options.display.max_rows = 30
import matplotlib.pyplot as plt
%matplotlib notebook
import gvar as gv
import fkfpi_lib as flib
print("python version:", sys.version)
#print("pandas version:", pd.__version__)
print("numpy  version:", np.__version__)
print("gvar   version:", gv.__version__)

python version: 3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
numpy  version: 1.16.1
gvar   version: 9.0.2


In [2]:
switches = dict()
switches['ensemble'] = [
    'a15m400','a12m400','a09m400',
    'a15m350','a12m350','a09m350',
    'a15m310','a12m310','a09m310',
    'a15m220','a12m220','a09m220','a12m220S','a12m220L',
    'a12m130','a15m130']
switches['ansatz'] = dict()
switches['ansatz']['FV'] = True # True turns on NLO FV correction
switches['ansatz']['type'] = 'xpt' # Type of fit: 'xpt' or 'MA'
switches['ansatz']['a2dm'] = 'individual' # avg or individual
switches['scale'] = 'PK' # PP, PK, KK, LamChi = 4 * pi * sqrt(FA * FB)
switches['debug'] = True

In [3]:
# Set priors for Xpt
priors = dict()
priors['L5'] = gv.gvar(0.0, 1.0)
priors['s4'] = gv.gvar(0.0, 1.0)
priors['c4'] = gv.gvar(0.0, 1.0)
priors['d4'] = gv.gvar(0.0, 1.0)
priors['e4'] = gv.gvar(0.0, 1.0)
priors['s6'] = gv.gvar(0.0, 1.0)
priors['sc6'] = gv.gvar(0.0, 1.0)
priors['sd6'] = gv.gvar(0.0, 1.0)
priors['se6'] = gv.gvar(0.0, 1.0)
# Physical parameters from PDG
phys_p = dict()
# http://pdg.lbl.gov/2016/tables/rpp2016-tab-mesons-light.pdf
phys_p['mpi'] = gv.gvar(134.8, 0.3) #FLAG 2017 (16)
phys_p['mka'] = gv.gvar(494.2 , 0.3) #FLAG 2017 (16)
phys_p['F0'] = gv.gvar(80,20) #FLAG use of F0 in SU(2) correction for FK/Fpi
phys_p['meta'] = gv.gvar(547.862,0.017) #PDG
# http://pdg.lbl.gov/2015/reviews/rpp2015-rev-pseudoscalar-meson-decay-cons.pdf
phys_p['Fpi'] = gv.gvar(130.2/np.sqrt(2), 1.7/np.sqrt(2)) #PDG fpi+ eq(16)
phys_p['Fka'] = gv.gvar(155.6/np.sqrt(2), 0.4/np.sqrt(2)) #PDG fK++ eq(16)
if switches['scale'] == 'PK':
    phys_p['Lchi'] = 4*np.pi*np.sqrt(phys_p['Fpi']*phys_p['Fka'])
elif switches['scale'] == 'PP':
    phys_p['Lchi'] = 4*np.pi*phys_p['Fpi']
elif switches['scale'] == 'KK':
    phys_p['Lchi'] = 4*np.pi*phys_p['Fka']
phys_p['aw0'] = 0
# FLAG FK/Fpi
flag_FKFpi = gv.gvar(1.1933, 0.0029)

In [4]:
data = h5.open_file('FK_Fpi_data.h5','r')
fmt_data = flib.format_h5_data(switches,data,priors)
data.close()

a15m400
{'mpi': 0.30283(31),'mk': 0.42723(27),'mss': 0.52194(22),'mju': 0.3596(23),'mjs': 0.4582(36),'mrs': 0.5538(18),'mru': 0.4711(20),'FK': 0.085012(91),'Fpi': 0.07934(12),'Fss': 0.089882(69)}
a12m400
{'mpi': 0.24346(16),'mk': 0.34341(14),'mss': 0.42073(13),'mju': 0.2679(16),'mjs': 0.3548(11),'mrs': 0.43206(70),'mru': 0.3621(12),'FK': 0.069762(68),'Fpi': 0.06492(11),'Fss': 0.074263(62)}
a09m400
{'mpi': 0.18114(15),'mk': 0.25522(13),'mss': 0.31327(11),'mju': 0.18737(90),'mjs': 0.2575(11),'mrs': 0.31589(62),'mru': 0.26040(82),'FK': 0.052260(67),'Fpi': 0.048341(75),'Fss': 0.056143(58)}
a15m350
{'mpi': 0.26475(30),'mk': 0.41371(28),'mss': 0.52010(23),'mju': 0.3311(35),'mjs': 0.4465(30),'mrs': 0.5529(17),'mru': 0.4592(38),'FK': 0.083674(85),'Fpi': 0.07686(11),'Fss': 0.089120(67)}
a12m350
{'mpi': 0.21397(20),'mk': 0.33309(16),'mss': 0.41971(13),'mju': 0.2404(13),'mjs': 0.3454(17),'mrs': 0.4319(11),'mru': 0.3519(28),'FK': 0.068504(74),'Fpi': 0.06292(14),'Fss': 0.073649(64)}
a09m350
{'mpi':

In [ ]:
data = pd.read_csv('./github_fkfpi_v2.csv')
hisq_params = pd.read_csv('./hisq_params.csv')
mixed_params = pd.read_csv('./github_mixed_v1.csv')
data.groupby('ensemble').describe()

In [ ]:
fmt_data = flib.format_data(switches,data,mixed_params,hisq_params,priors)
#print(fmt_data)
print(fmt_data['y'])

In [ ]:
jr = ['ju','ru','js','rs']
mrks = ['s','o','d','H']
shift = [-0.003,-0.001,0.001,0.003]
fig = plt.figure('w0_DMix',figsize=(7,4.326237))
ax = plt.axes([0.15,0.15,0.8,0.8])
colors ={'a15m400':'#ec5d57','a15m350':'#ec5d57','a15m310':'#ec5d57','a15m220':'#ec5d57','a15m130':'#ec5d57',
        'a12m400':'#70bf41','a12m350':'#70bf41','a12m310':'#70bf41','a12m220':'#70bf41','a12m130':'#70bf41','a12m220S':'#70bf41','a12m220L':'#70bf41',
        'a09m400':'#51a7f9','a09m350':'#51a7f9','a09m310':'#51a7f9','a09m220':'#51a7f9'}
for i,e in enumerate(switches['ensemble']):
    epi = fmt_data['p']['mpi'][i]/fmt_data['x']['Lchi'][i]
    for j,mix in enumerate(jr):
        wMix = fmt_data['p']['wMix_'+mix][i]
        ax.errorbar(epi.mean+shift[j],wMix.mean,yerr=wMix.sdev,color=colors[e],marker=mrks[j],elinewidth=1,capsize=3,mfc='none')

In [ ]:
fit = flib.fit_data(switches,fmt_data,phys_p)
print(fit.format(maxline=True))

In [ ]:
fig = plt.figure('FK/Fpi',figsize=(7,4.326237))
ax = plt.axes([0.15,0.15,0.8,0.8])
colors ={'a15m400':'#ec5d57','a15m350':'#ec5d57','a15m310':'#ec5d57','a15m220':'#ec5d57','a15m130':'#ec5d57',
        'a12m400':'#70bf41','a12m350':'#70bf41','a12m310':'#70bf41','a12m220':'#70bf41','a12m130':'#70bf41','a12m220S':'#70bf41','a12m220L':'#70bf41',
        'a09m400':'#51a7f9','a09m350':'#51a7f9','a09m310':'#51a7f9','a09m220':'#51a7f9'}
for i,e in enumerate(switches['ensemble']):
    epi = fmt_data['p']['mpi'][i]/fmt_data['x']['Lchi'][i]
    print('%8s %s %s' %(e,str(fmt_data['y'][i]),str(fit.fcn(fit.x,fit.p)[i])))
    ax.errorbar(epi.mean,fmt_data['y'][i].mean,yerr=fmt_data['y'][i].sdev,color=colors[e],marker='o',elinewidth=1,capsize=3,mfc='none')
    ax.errorbar(epi.mean+.005,fit.fcn(fit.x,fit.p)[i].mean,yerr=fit.fcn(fit.x,fit.p)[i].sdev,color=colors[e],marker='s',elinewidth=1,capsize=3,mfc='none')
fkp = flib.fkfpi_phys(phys_p,fit)
ax.errorbar((phys_p['mpi']/(4.*np.pi*phys_p['Fpi'])).mean,fkp.mean,yerr=fkp.sdev,color='magenta',marker='*',elinewidth=1,capsize=3,mfc='none')
ax.errorbar((phys_p['mpi']/(4.*np.pi*phys_p['Fpi'])).mean,1.193,yerr=0.003,color='black',marker='*',elinewidth=1,capsize=3,mfc='none')
plt.savefig('FKFpi_vs_epi.pdf',transparent=True)